In [ ]:
import pandas as pd
import numpy as np
import glob

from google.colab import drive
drive.mount('/content/drive')

file_paths=glob.glob('/content/drive/MyDrive/WDL/police_complaints/*.csv')

file_paths=glob.glob('/content/drive/MyDrive/WDL/police_complaints/*.csv')

concat_data = []
for file in file_paths:
      df = pd.read_csv(file, sep=',')
      df['Timestamp'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
      concat_data.append(df)
df_final = pd.concat(concat_data)

In [ ]:
filter = ['Facilities disturbances', 'Disturbing noises', 'Youth aggregation']

df_filtered = df_final.loc[df_final['Criminal sub-category'].isin(filter)].reset_index()

#San Salvario district 8

df_filtered = df_filtered[df_filtered['District'] == 8.0 ]

df_filtered['Localization'] = df_filtered['Localization'].str.lower().str.strip()

In [ ]:
file_paths=glob.glob('/content/drive/MyDrive/WDL/noise_data/*.csv')


sensor_list = pd.read_csv('/content/drive/MyDrive/WDL/noise_sensor_list.csv', sep = ';')
sensor_list['Sensor_ID'] = ['C1', 'C2', 'C3', 'C4', 'C5']

sensor_list['Lat'] = sensor_list['Lat'].str.replace(',', '.').astype(float)
sensor_list['Long'] = sensor_list['Long'].str.replace(',', '.').astype(float)


def load_noise_data(file_paths, sensor_list):
    """
    Function for loading noise data into the correct format
    """
    concat_data = []
    for file in file_paths:
        df = pd.read_csv(file, header=8, sep=';')
        df = df.melt(id_vars=['Data', 'Ora'])
        df['Timestamp'] = pd.to_datetime(df['Data'] + ' ' + df['Ora'])
        df.columns = ['Date', 'Time', 'Sensor_ID', 'Intensity', 'Timestamp']

        concat_data.append(df)

    concat_df = pd.concat(concat_data)

    output = concat_df.merge(sensor_list, on=['Sensor_ID'])
    return output[['Timestamp', 'Sensor_ID', 'Intensity', 'address', 'Lat', 'Long']]

data = load_noise_data(file_paths, sensor_list)

In [5]:
localization_address_mapping = {
   'principe tommaso/(via)':  'Via Principe Tommaso, 18bis Torino',
   'baretti/giuseppe (via)': 'Via Principe Tommaso angolo via Baretti Torino',
   'marconi/guglielmo (corso)' : 'Corso Marconi, 27 Torino',
   'saluzzo/(largo)': 'Largo Saluzzo Torino',
   'saluzzo/(via)': 'Via Saluzzo, 26 Torino'
}

In [ ]:
df_filtered['address'] = df_filtered['Localization'].map(localization_address_mapping)
tmp = df_filtered.merge(data, on=['address'])

tmp['Timestamp_x'] = pd.to_datetime(tmp['Timestamp_x'])
tmp['Timestamp_y'] = pd.to_datetime(tmp['Timestamp_y'])

tmp['day_difference'] = (tmp['Timestamp_x'] - tmp['Timestamp_y']).dt.days

def label (row):
   if row['day_difference'] == 1 or row['day_difference'] == 0 :
      return 1
   else:
      return 0

tmp['label'] = tmp.apply(label, axis=1)